In [1]:
import torch

In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

In [3]:
training_data = datasets.FashionMNIST(
    root = 'FashionMNIST/',
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



In [4]:
test_data = datasets.FashionMNIST(
    root = 'FashionMNIST/',
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    # N : Mini-Batch의 크기, C : Chanel의 개수(RGB 이거나 흑백정도), Height, Width
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # RGB 이므로 평탄화 해줌
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss() # 손실함수
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) # 최적화 방법 적용

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):# epoch 마다 가중치 갱신
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        # 무조건 optimizer.zero_grad()해줘야 함
        optimizer.zero_grad()
        # 손실함수 미분
        loss.backward()
        # 가중치 갱신
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.927452  [    0/60000]
loss: 0.977593  [ 6400/60000]
loss: 0.758446  [12800/60000]
loss: 0.935013  [19200/60000]
loss: 0.816183  [25600/60000]
loss: 0.833223  [32000/60000]
loss: 0.903456  [38400/60000]
loss: 0.862858  [44800/60000]
loss: 0.881394  [51200/60000]
loss: 0.844757  [57600/60000]
Test Error: 
 Accuracy: 68.6%, Avg loss: 0.840861 

Epoch 2
-------------------------------
loss: 0.867709  [    0/60000]
loss: 0.930131  [ 6400/60000]
loss: 0.701994  [12800/60000]
loss: 0.890223  [19200/60000]
loss: 0.778761  [25600/60000]
loss: 0.786301  [32000/60000]
loss: 0.865558  [38400/60000]
loss: 0.830937  [44800/60000]
loss: 0.843366  [51200/60000]
loss: 0.812063  [57600/60000]
Test Error: 
 Accuracy: 69.6%, Avg loss: 0.805145 

Epoch 3
-------------------------------
loss: 0.818882  [    0/60000]
loss: 0.892458  [ 6400/60000]
loss: 0.658530  [12800/60000]
loss: 0.855559  [19200/60000]
loss: 0.750433  [25600/60000]
loss: 0.750276  [32000/600